In [1]:
%pip install 'scipy>=1.8'
%pip install 'networkx<2.7'
%pip install igraph
%pip install pandas == 1.5.3
%pip install infomap


--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pandas as pd
from tqdm import tqdm

from matplotlib import pyplot
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier


import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


# ML models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier

from collections import Counter

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# Speed up hyperparameter tuning
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import networkx as nx
import infomap


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# PATH = './kaggle/input/healthcare-provider-fraud-detection-analysis/'


In [ ]:
inpatients = pd.read_csv('Train_Inpatientdata-1542865627584.csv')
outpatients = pd.read_csv('Train_Outpatientdata-1542865627584.csv')
beneficiaries = pd.read_csv('Train_Beneficiarydata-1542865627584.csv')

labels = pd.read_csv('Train-1542865627584.csv')


In [ ]:
inpatients.head()


In [ ]:
labels.head()


In [ ]:
inpatients.columns


In [ ]:
outpatients.columns


In [ ]:
beneficiaries.head()


In [ ]:
beneficiaries.info()


In [ ]:
beneficiaries.columns


In [ ]:
labels.columns


In [ ]:
df1 = inpatients[['BeneID', 'ClaimID', 'Provider',
                  'InscClaimAmtReimbursed', 'AttendingPhysician', 'DeductibleAmtPaid']]
df2 = outpatients[['BeneID', 'ClaimID', 'Provider',
                   'InscClaimAmtReimbursed', 'AttendingPhysician', 'DeductibleAmtPaid']]
df3 = beneficiaries[['BeneID', 'Gender', 'Race', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
                     'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt']]


df = pd.concat([df1, df2])

df = df.set_index('Provider').join(labels.set_index('Provider')).reset_index(
).set_index('BeneID').join(df3.set_index('BeneID')).reset_index()

df['PotentialFraud'] = df['PotentialFraud'].replace(
    "No", 0).replace("Yes", 1).astype(int)

df.head()


In [ ]:
# Providers: start with 1
#AttendingPhysician: starts with 2


In [ ]:
df['AttendingPhysician'].str.removeprefix("PHY").astype(float).min()


In [ ]:
df['Provider'] = df['Provider'].str.removeprefix("PRV").astype(int) + 1_000_000
df['AttendingPhysician'] = df['AttendingPhysician'].str.removeprefix(
    "PHY").fillna(0).astype(int) + 2_000_000
df['BeneID'] = df['BeneID'].str.removeprefix(
    "BENE").fillna(0).astype(int) + 3_000_000

df


In [ ]:
df.info()


In [ ]:
df.head()


In [ ]:
#Plotting graph now

In [ ]:
df.shape


In [ ]:
#trying collusion between provider & physician
source = 'Provider'
target = 'AttendingPhysician'

G = nx.from_pandas_edgelist(df, source=source, target=target)


In [ ]:
import plotly.graph_objs as go

# Consider subgraph for plotting
df_plotting = df.sample(n=1_000, random_state=1)

H = nx.from_pandas_edgelist(df_plotting, source=source, target=target)
pos = nx.random_layout(H)

# Create Edges
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#010203'),
    hoverinfo='none',
    mode='lines')

for edge in H.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    mode='markers',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=False,
        color=[],
        size=12,
        colorbar=dict(
            thickness=35,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in H.nodes():
    x, y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

# add color to node points
for node, adjacencies in enumerate(H.adjacency()):
    node_trace['marker']['color'] += tuple([len(adjacencies[1])])
    node_info = str(adjacencies[0])

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                title='<br>Network Graph of Provider & Physician \n\n',
                titlefont=dict(size=16),
                showlegend=False,
                margin=dict(b=20, l=5, r=5, t=40),
                annotations=[dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002)],
                xaxis=dict(showgrid=False, zeroline=False,
                           showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()


In [ ]:
# defining a dictionary of graph features
nodes_info_dict = {
    # 'closeness_centrality': nx.closeness_centrality,
    'eigenvector_centrality': nx.eigenvector_centrality_numpy,
    'pagerank': nx.pagerank
}

columns_with_node_infos = ['degree'] + list(nodes_info_dict.keys())

nodes_info = pd.DataFrame.from_dict(dict(nx.degree(G)), orient='index').rename(
    columns={0: 'degree'}).reset_index()

# computing graph features for each node
for info, fun in nodes_info_dict.items():
    temp = pd.DataFrame.from_dict(fun(G), orient='index').rename(columns = {0 : info}).reset_index()
    nodes_info = nodes_info.merge(temp, on='index')

nodes_info = nodes_info.rename(columns = {'index': 'Physician'})

# adding graph features to the dataframe
df_enriched = df.merge(nodes_info, left_on = 'Provider',
                           right_on='Physician', how='left').drop('Physician', axis=1)
df_enriched.rename(columns = {k:'Provider_'+k for k in columns_with_node_infos}, inplace = True)

df_enriched = df_enriched.merge(nodes_info, left_on = 'AttendingPhysician',
                           right_on='Physician', how='left').drop('Physician', axis=1)
df_enriched.rename(columns = {k:'AttendingPhysician_'+k for k in columns_with_node_infos}, inplace = True)

In [ ]:
df_enriched.head()


In [ ]:
df_enriched.shape


In [ ]:
# The graph below shows the correlation between the target PotentialFraud and the graph features.


In [ ]:
import seaborn as sns

corr_data = df_enriched[['PotentialFraud',
                         'AttendingPhysician',
                         'Provider_degree',
                         #                 'Provider_closeness_centrality',
                         'Provider_eigenvector_centrality',
                         'Provider_pagerank',
                         'AttendingPhysician_degree',
                         #                 'AttendingPhysician_closeness_centrality',
                         'AttendingPhysician_eigenvector_centrality',
                         'AttendingPhysician_pagerank']]

# Compute the correlation matrix
corr = corr_data.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
def findCommunities(G):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id and return number of communities found.
    """
    im = infomap.Infomap(two_level=True, silent=True)

    print("Building Infomap network from a NetworkX graph...")
    for e in G.edges():
        im.addLink(*e)

    print("Find communities with Infomap...")
    im.run()

    print(
        f"Found {im.num_top_modules} modules with codelength {im.codelength:.8f} bits")

    communities = {}
    for node, module in im.modules:
        communities[node] = module

    nx.set_node_attributes(G, communities, 'community')

    return G


def drawNetwork(G):
    # position map
    pos = nx.spectral_layout(I)
    # community ids
    communities = [v for k, v in nx.get_node_attributes(
        I, 'community').items()]
    numCommunities = max(communities) + 1
    # color map from http://colorbrewer2.org/
    cmapLight = colors.ListedColormap(
        ['#a6cee3', '#b2df8a', '#fb9a99', '#fdbf6f', '#cab2d6'], 'indexed', numCommunities)
    cmapDark = colors.ListedColormap(
        ['#1f78b4', '#33a02c', '#e31a1c', '#ff7f00', '#6a3d9a'], 'indexed', numCommunities)

    # Draw edges
    nx.draw_networkx_edges(I, pos)


# Draw nodes
    nodeCollection = nx.draw_networkx_nodes(I,
                                            pos=pos,
                                            node_size=1,
                                            node_color=communities,
                                            cmap=cmapLight
                                            )
    # Set node border color to the darker shade
    darkColors = [cmapDark(v) for v in communities]
    nodeCollection.set_edgecolor(darkColors)

    plt.axis('off')
    plt.xlim(np.vstack(list(pos.values()))[:, 0].min(
    ) + 0.01, np.vstack(list(pos.values()))[:, 0].max())
    plt.ylim(np.vstack(list(pos.values()))[:, 1].min(
    ), np.vstack(list(pos.values()))[:, 1].max())
    plt.show()


In [ ]:
G = findCommunities(G)


In [ ]:
# plotting the communities as a graph


In [ ]:
I = G.subgraph(list(H.nodes()))

drawNetwork(I)


In [ ]:
df_communities = pd.DataFrame([[k, v] for k, v in nx.get_node_attributes(
    G, 'community').items()], columns=["AttendingPhysician", "AttendingPhysician_cluster"])
df_communities.head(2)


In [ ]:
df_communities.info()


In [ ]:
df_enriched = df_enriched.set_index('AttendingPhysician').join(
    df_communities.set_index('AttendingPhysician'), how="left", rsuffix='_comm').reset_index()

df_enriched.head(2)


In [ ]:
#Setting up training
 #I split the dataset into 2: a training set, 
#representing 80% of the initial dataset, and 
#a validation set. 
#For every scenario, I tested multiple 
#classification algorithms and picked the best one based on the AUC on the training set 
#using a 3-fold cross-validation approach.

In [ ]:
df_enriched.info()


In [ ]:
classes = df_enriched['PotentialFraud'].to_numpy()

plt.hist(classes)
plt.title("Showing the class imbalance")
plt.show()

print('The Class Imbalance: %s' % Counter(classes))


In [ ]:
df_enriched.head()


In [ ]:
df_enriched.columns


In [ ]:
# Splitting feature data from label data
X, y = df_enriched[['InscClaimAmtReimbursed',
                    'DeductibleAmtPaid',
                    'Gender',
                    'Race',
                    'NoOfMonths_PartACov',
                    'NoOfMonths_PartBCov',
                    'IPAnnualReimbursementAmt',
                    'IPAnnualDeductibleAmt',
                    'OPAnnualReimbursementAmt',
                    'OPAnnualDeductibleAmt',
                    'Provider_degree',
                    #  'Provider_closeness_centrality',
                    'Provider_eigenvector_centrality',
                    'Provider_pagerank',
                    'AttendingPhysician_degree',
                    #  'AttendingPhysician_closeness_centrality',
                    'AttendingPhysician_eigenvector_centrality',
                    'AttendingPhysician_pagerank',
                    'AttendingPhysician_cluster']], df_enriched['PotentialFraud']

print("Original shapes: ", "X:", X.shape, " y:", y.shape)


In [ ]:
X = X.fillna(0)


In [ ]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=69)
print(f"Shapes after splitting:\n\nX_train: {X_train.shape}, y_train: {y_train.shape}\
      \nX_test: {X_test.shape}, y_test: {y_test.shape}")


In [ ]:
#Training a model

In [ ]:
accuracy = []
feature_names = ["Baseline", "Graph Features",
                 "Graph Features with Community Detection"]
features = [
    # Baseline features
    ['InscClaimAmtReimbursed',
     'DeductibleAmtPaid',
     'Gender',
     'Race',
     'NoOfMonths_PartACov',
     'NoOfMonths_PartBCov',
     'IPAnnualReimbursementAmt',
     'IPAnnualDeductibleAmt',
     'OPAnnualReimbursementAmt',
     'OPAnnualDeductibleAmt'],
    # Baseline + Graph Features
    ['InscClaimAmtReimbursed',
     'DeductibleAmtPaid',
     'Gender',
     'Race',
     'NoOfMonths_PartACov',
     'NoOfMonths_PartBCov',
     'IPAnnualReimbursementAmt',
     'IPAnnualDeductibleAmt',
     'OPAnnualReimbursementAmt',
     'OPAnnualDeductibleAmt',
     'Provider_degree',
     #  'Provider_closeness_centrality',
     'Provider_eigenvector_centrality',
     'Provider_pagerank',
     'AttendingPhysician_degree',
     #  'AttendingPhysician_closeness_centrality',
     'AttendingPhysician_eigenvector_centrality',
     'AttendingPhysician_pagerank'],
    # Baseline + Graph Features + Community Detection
    ['InscClaimAmtReimbursed',
     'DeductibleAmtPaid',
     'Gender',
     'Race',
     'NoOfMonths_PartACov',
     'NoOfMonths_PartBCov',
     'IPAnnualReimbursementAmt',
     'IPAnnualDeductibleAmt',
     'OPAnnualReimbursementAmt',
     'OPAnnualDeductibleAmt',
     'Provider_degree',
     #  'Provider_closeness_centrality',
     'Provider_eigenvector_centrality',
     'Provider_pagerank',
     'AttendingPhysician_degree',
     #  'AttendingPhysician_closeness_centrality',
     'AttendingPhysician_eigenvector_centrality',
     'AttendingPhysician_pagerank',
     'AttendingPhysician_cluster']
]

hyper_parameter_grids_RFC = [
    {  # Grid 1: No regularization
        "randomforestclassifier__criterion": ['gini'],
        "randomforestclassifier__max_depth": [10, 20, 50, 100, 250, 300, 500],
        "randomforestclassifier__min_samples_split": [2, 3, 5, 10, 20, 30],
    },
    {  # Grid 2: L2 regularization
        "randomforestclassifier__criterion": ['entropy'],
        "randomforestclassifier__max_depth": [10, 20, 50, 100, 250, 300, 500],
        "randomforestclassifier__min_samples_split": [2, 3, 5, 10, 20, 30],

    },
]
pipeline_RFC = make_pipeline(
    StandardScaler(), RandomForestClassifier(random_state=69))


for feature in features:
    print("*" * 100)
    print("# Tuning hyper-parameters for accuracy")
    print("*" * 100)
    print()

    # This performs gridsearch, evaluating each set of hyper-parameters using k-fold
    # cross validation.
    clf = HalvingGridSearchCV(
        pipeline_RFC, hyper_parameter_grids_RFC, scoring="accuracy", cv=4, n_jobs=-1)

    X_train_subset = X_train[feature]
    X_test_subset = X_test[feature]

    clf.fit(X_train_subset, y_train)

    acc = round(clf.best_estimator_.score(X_test_subset, y_test) * 100, 2)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Best score on development set:")
    print()

    print(f"Accuracy: {acc}")
    accuracy.append(acc)